In [59]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config

In [60]:
import random
import json
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

In [61]:
client = OpenAI(api_key=config.Config.OPENAI_API_KEY)
qdrant_client = QdrantClient(url='localhost')
embedding_model = SentenceTransformer("BAAI/bge-m3")
collection_name = 'ragscout_players'

In [62]:
with open("../../data/player_summaries.json", "r", encoding="utf-8") as f:
    players_summaries = json.load(f)
    
df_player_summaries = pd.DataFrame([
    {'player': player.split('(')[0].strip(), 'team': player.split('(')[1].replace(')', '').strip(), 'summary': summary} for player, summary in players_summaries.items()
])
df_player_summaries = df_player_summaries.reset_index() 
df_player_summaries.head()

,index,player,team,summary
0,0,Ben White,Arsenal,Le joueur évolue principalement en tant que dé...
1,1,Bukayo Saka,Arsenal,Le joueur évolue principalement en tant qu'att...
2,2,David Raya,Arsenal,Le joueur évolue principalement au poste de ga...
3,3,Declan Rice,Arsenal,Le joueur évolue principalement au poste de mi...
4,4,Ethan Nwaneri,Arsenal,Le joueur évolue principalement en tant qu'att...


In [63]:
sample_players = df_player_summaries['player'].unique().tolist()
selected_players = random.sample(sample_players, 100)

df_selected_players = df_player_summaries[df_player_summaries['player'].isin(selected_players)]
df_selected_players.head(5)

,index,player,team,summary
11,11,Martin Ødegaard,Arsenal,Le joueur évolue principalement au poste de mi...
27,27,Marco Asensio,Aston Villa,Le joueur évolue principalement au poste de mi...
41,41,Illia Zabarnyi,Bournemouth,Le joueur évolue principalement en tant que dé...
52,52,Ethan Pinnock,Brentford,Le joueur évolue principalement en tant que dé...
60,60,Nathan Collins,Brentford,Le joueur évolue principalement en tant que dé...


In [69]:
evaluations = []

for _, row in df_selected_players.head(5).iterrows():
    player_key = f"{row['player']} ({row['team']})"
    
    prompt = f"""
        Tu es recruteur dans un club professionnel.

        Tu viens de lire un rapport de scouting détaillé décrivant le style de jeu, les points forts, et les axes d’amélioration d’un joueur.

        Ton objectif : formuler **une requête courte, naturelle et spécifique** que taperait un recruteur dans un moteur de recherche intelligent pour retrouver exactement **ce profil de joueur**.

        Exemples:
            -   Défenseur axial dominant dans les duels et solide à la relance
	        -   Milieu récupérateur intense avec bon jeu long et volume de course élevé
	        -   Ailier droit rapide, percutant et impliqué dans le pressing haut
	        -   Milieu relayeur capable de se projeter et précis dans les passes verticales
         
        ---

        🔍 Ta requête doit :
        - Être une **seule phrase fluide et concise**
        - Mentionner le **poste (ou sous-rôle)** et **2 ou 3 caractéristiques clés**
        - Utiliser des termes **précis et différenciants** (ex. : “pressing intense”, “relance propre”, “jeu entre les lignes”, “présence aérienne”, “percussion côté gauche”)
        - Ne pas être vague : évite les termes génériques comme “bon techniquement” ou “polyvalent”
        - ❌ Ne fais **aucune référence** au nom, au club, à la nationalité ou à la ligue du joueur

        Voici le résumé à analyser :

        \"\"\"{row['summary']} \"\"\"

        Donne uniquement la requête, sans guillemets, sans commentaire.
    """
    
    response = client.chat.completions.create(
        model = config.Config.OPENAI_MODEL,
        messages = [
            {"role": "system", "content": "Tu es un recruteur football très concis"},
            {"role": "user", "content": prompt}
        ],
        temperature = 0.4,
        max_tokens = 50
    )
    
    query = response.choices[0].message.content.strip()
    
    evaluations.append({
        'query': query,
        'expected_player': row['player']
    })

In [70]:
evaluations[:5]

[{'query': 'Milieu créatif avec passes clés, bonne condition physique et implication dans le pressing.',
  'expected_player': 'Martin Ødegaard'},
 {'query': 'Milieu de terrain créatif avec bonne présence physique, capacité à progresser et implication dans le pressing.',
  'expected_player': 'Marco Asensio'},
 {'query': 'Défenseur central solide, impliqué dans le pressing, avec bonne relance et volume de jeu élevé.',
  'expected_player': 'Illia Zabarnyi'},
 {'query': 'Défenseur central solide avec bonne présence dans les duels et relance précise.',
  'expected_player': 'Ethan Pinnock'},
 {'query': 'Défenseur central solide, engagé dans les duels et efficace dans les interceptions.',
  'expected_player': 'Nathan Collins'}]

In [71]:
def evaluate_retrieval(eval_queries, k=50):
    hit_at_k = []
    mrr = []
    not_found_cases = []

    for q in tqdm(eval_queries, desc="Évaluation des requêtes"):
        query = q["query"]
        expected = q["expected_player"].lower()

        # Embedding de la requête
        query_vector = embedding_model.encode(query).tolist()

        # Recherche Qdrant
        results = qdrant_client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=k
        )

        found = False
        reciprocal_rank = 0

        for idx, hit in enumerate(results):
            player_name = hit.payload.get("player", "").lower()
            if player_name == expected:
                found = True
                reciprocal_rank = 1 / (idx + 1)
                break

        hit_at_k.append(1 if found else 0)
        mrr.append(reciprocal_rank)

        if not found:
            not_found_cases.append({
                "query": query,
                "expected": expected,
                "results": [hit.payload.get("player") for hit in results]
            })

    score = {
        f"Hit@{k}": round(sum(hit_at_k) / len(hit_at_k), 3),
        "MRR": round(sum(mrr) / len(mrr), 3),
        "not_found": not_found_cases
    }
    return score

In [72]:
evaluation_results = evaluate_retrieval(evaluations)
evaluation_results

Évaluation des requêtes:   0%|          | 0/5 [00:00<?, ?it/s]/var/folders/x9/t16jfwq91z735wm10ntxvh780000gq/T/ipykernel_70048/2858153860.py:14: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(
Évaluation des requêtes: 100%|██████████| 5/5 [00:01<00:00,  4.97it/s]


{'Hit@50': 0.2,
 'MRR': 0.2,
 'not_found': [{'query': 'Milieu de terrain créatif avec bonne présence physique, capacité à progresser et implication dans le pressing.',
   'expected': 'marco asensio',
   'results': ['Jude Bellingham',
    'Hakan Çalhanoğlu',
    'Jean-Ricner Bellegarde',
    'Martin Ødegaard',
    'Yorbe Vertessen',
    'Luka Modrić',
    'Declan Rice',
    'Joshua Kimmich',
    'Patrick Wimmer',
    'Nikola Vlašić',
    'Yannick Gerhardt',
    'Kevin De Bruyne',
    'Exequiel Palacios',
    'Bilal El Khannouss',
    'Elliot Anderson',
    'Conor Chaplin',
    'Giuliano Simeone',
    'Fisayo Dele-Bashiru',
    'Luis Rioja',
    'Mario Götze',
    'Carlos Martín',
    'Dwight McNeil',
    'Tomáš Souček',
    'Mario Pašalić',
    'Valentin Mihaila',
    'Jens Odgaard',
    'Edoardo Bove',
    'Dany Mota',
    'Ludovic Blas',
    'Junior Messias',
    'Florian Wirtz',
    'Julian Brandt',
    'Elias Saad',
    'Boulaye Dia',
    'Hákon Arnar Haraldsson',
    'Domingos Andr